# Modular Neural Networks

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
# read data 
option_input_name = 'option.pkl'
option = pd.read_pickle(option_input_name)
option.head()

,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,...,dividend_rate,BS_realized_vol_price,BS_realized_vol_square_error,moneyness,garch_recursive_vol,garch_Rolling_vol,BS_garch_rolling_price,BS_garch_rolling_square_error,BS_implied_vol_price,VG_price
date,,,,,,,,,,,,,,,,,,,,,
2000-08-07,2000-09-16,call,1425.0,73.625,75.625,12,5432,0.185298,0.774554,0.003385,...,0.009039,77.262369,0.001249,1.038119,0.20733,0.20733,74.014884,0.000067,70.490622,NaN
2000-08-07,2000-10-21,put,1325.0,7.250,8.250,10,140,0.225634,-0.104144,0.001210,...,0.009039,9.909709,0.077658,1.116468,0.20733,0.20733,7.344116,0.002743,9.724810,NaN
2000-08-07,2000-10-21,call,1475.0,58.875,60.875,100,625,0.185426,0.584703,0.003171,...,0.009039,64.615685,0.006269,1.002929,0.20733,0.20733,59.398691,0.000063,53.583519,NaN
2000-08-07,2000-08-19,put,1525.0,47.500,49.500,71,728,0.199673,-0.803766,0.005656,...,0.009039,53.423762,0.010306,0.970046,0.20733,0.20733,51.828331,0.004709,51.230861,NaN
2000-08-07,2000-08-19,put,1470.0,13.125,14.375,57,3390,0.195372,-0.397165,0.008059,...,0.009039,19.550511,0.177962,1.006340,0.20733,0.20733,17.486737,0.073855,16.233676,NaN


In [3]:
# clean data
option = option[option.best_bid>1]
option = option[option.best_offer>1]
option = option.dropna()

# extract calls and puts
call_option = option[option['cp_flag']=='call'].copy()
put_option = option[option['cp_flag']=='put'].copy()

In [4]:
# create moneyness_class for call option
call_option['moneyness_class'] = np.where(call_option['moneyness']>1.05,'itm','atm')
call_option['moneyness_class'] = np.where(call_option['moneyness']<0.97,'otm',call_option['moneyness_class'])

# create tau_class for call option
call_option['tau_class'] = np.where(call_option['tau']>0.2,'longterm','midterm')
call_option['tau_class'] = np.where(call_option['tau']<0.1,'shortterm',call_option['tau_class'])

# create moneyness_class for put option
put_option['moneyness_class'] = np.where(put_option['moneyness']<0.97,'itm','atm')
put_option['moneyness_class'] = np.where(put_option['moneyness']>1.05,'otm',put_option['moneyness_class'])

# create tau_class for put option
put_option['tau_class'] = np.where(put_option['tau']>0.2,'longterm','midterm')
put_option['tau_class'] = np.where(put_option['tau']<0.1,'shortterm',put_option['tau_class'])


In [5]:
# general function
def MNN_pricing(option_data,moneyness_class,tau_class,nodes_each_layer,
                     activation_functions,inverse_transform,output_df):
    data = option_data[(option_data['moneyness_class']==moneyness_class) & (option_data['tau_class']==tau_class)]
    years_performance = []
    years_data = pd.DataFrame()
    years = list(data.index.year.unique()) 
    years.remove(2016) # too few samples
    yearly_data = [data[data.index.year == i] for i in years]
    for yearwise_data in yearly_data: #different years
        yearwise_data_train = yearwise_data.iloc[:int(len(yearwise_data)/2),:]
        yearwise_data_validation = yearwise_data.iloc[int(len(yearwise_data)/2):int(len(yearwise_data)*3/4),:]
        yearwise_data_test= yearwise_data.iloc[int(len(yearwise_data)*3/4):,:]
        
        # build NN model:
        model = Sequential()
        model.add(Dense(nodes_each_layer[0],activation = activation_functions[0],input_shape = (2,)))
        model.add(Dropout(0.2))
        model.add(Dense(nodes_each_layer[1],activation = activation_functions[1]))
        model.add(Dropout(0.2))
        model.add(Dense(nodes_each_layer[2],activation = activation_functions[2]))
        model.compile(loss='mean_squared_error',optimizer=RMSprop(),metrics=['mae'])
                
        # normalize
        sc1 = StandardScaler() # fit training features
        sc2 = StandardScaler() # fit training target, inverse transform for test prediction
        sc3 = StandardScaler() # fit validation features and/or target (shared without reuse conflict)
        print('----------year = ',yearwise_data.index.year[0],'-----------')
        
        # early stopping
        es = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=3,verbose=2, mode='auto') 
        
        # fit
        if inverse_transform == False: 
            history = model.fit(sc1.fit_transform(yearwise_data_train[['moneyness','tau']]),yearwise_data_train[['last']],batch_size=50,epochs=100,verbose=2,validation_data=(sc3.fit_transform(yearwise_data_validation[['moneyness','tau']]),yearwise_data_validation[['last']]),callbacks=[es])
            
            # percentage mean-squared-error:    
            prediction = model.predict(sc1.transform(yearwise_data_test[['moneyness','tau']]))
            yearwise_data_test_copy = yearwise_data_test.copy()
            yearwise_data_test_copy['MNN_price'] = prediction  # inverse transform
            yearwise_data_test_copy['MNN_square_error'] = ((yearwise_data_test['last']-yearwise_data_test_copy['MNN_price'])/yearwise_data_test['last'])**2
            
            years_performance.append(yearwise_data_test_copy['MNN_square_error'].mean())    
            years_data = years_data.append(yearwise_data_test_copy) 
        
        elif inverse_transform == True: # fit_transform on target for later inverse_transform
            history = model.fit(sc1.fit_transform(yearwise_data_train[['moneyness','tau']]),sc2.fit_transform(yearwise_data_train[['last']]),batch_size=50,epochs=100,verbose=2,validation_data=(sc3.fit_transform(yearwise_data_validation[['moneyness','tau']]),sc3.fit_transform(yearwise_data_validation[['last']])),callbacks=[es])
            
            # percentage mean-squared-error:    
            prediction = model.predict(sc1.transform(yearwise_data_test[['moneyness','tau']]))
            yearwise_data_test_copy = yearwise_data_test.copy()
            yearwise_data_test_copy['MNN_price'] = sc2.inverse_transform(prediction)  # inverse transform
            yearwise_data_test_copy['MNN_square_error'] = ((yearwise_data_test['last']-yearwise_data_test_copy['MNN_price'])/yearwise_data_test['last'])**2
            
            years_performance.append(yearwise_data_test_copy['MNN_square_error'].mean()) 
            years_data = years_data.append(yearwise_data_test_copy) 
        
    # aggregete
    performance.append(years_performance)
    output_df = output_df.append(years_data)
    print('percentage mean square error of',tau_class,moneyness_class,'call:',np.mean(years_performance))
    return output_df

In [ ]:
# container
performance = []
MNN_output = pd.DataFrame()

# test
MNN_output = MNN_pricing(call_option,'itm','shortterm',[10,5,1],['selu','relu','linear'],True,MNN_output)
MNN_output = MNN_pricing(call_option,'itm','midterm',[10,5,1],['selu','relu','linear'],True,MNN_output)
MNN_output = MNN_pricing(call_option,'itm','longterm',[10,5,1],['selu','relu','linear'],True,MNN_output)
MNN_output = MNN_pricing(call_option,'atm','shortterm',[10,5,1],['relu','relu','linear'],False,MNN_output)
MNN_output = MNN_pricing(call_option,'atm','midterm',[10,5,1],['relu','relu','linear'],False,MNN_output)
MNN_output = MNN_pricing(call_option,'atm','longterm',[10,5,1],['relu','relu','linear'],False,MNN_output)
MNN_output = MNN_pricing(call_option,'otm','shortterm',[10,5,1],['relu','relu','linear'],False,MNN_output)
MNN_output = MNN_pricing(call_option,'otm','midterm',[10,5,1],['relu','relu','linear'],False,MNN_output)
MNN_output = MNN_pricing(call_option,'otm','longterm',[10,5,1],['relu','relu','linear'],False,MNN_output)

In [ ]:
# merge
option = option.merge(MNN_output,how = 'left') 
option.to_pickle('option_addMNN.pickle')